In [2]:
import pandas as pd
#import matplotlib.pyplot as plt
import string
import re
import nltk
import numpy as np
from nltk.corpus import stopwords
from nltk.tokenize import RegexpTokenizer
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import GaussianNB
from sklearn.naive_bayes import MultinomialNB
from sklearn.ensemble import RandomForestRegressor
import sklearn.metrics
#import lime
import sklearn.ensemble
from __future__ import print_function
#from lime import lime_text
from sklearn.pipeline import make_pipeline
#from lime.lime_text import LimeTextExplainer
#from lime.lime_tabular import LimeTabularExplainer
#import imgkit
#from datetime import datetime
import warnings
warnings.filterwarnings("ignore")

In [3]:
#download nltk stuff
nltk.download('stopwords')
nltk.download('punkt')

[nltk_data] Error loading stopwords: <urlopen error [SSL:
[nltk_data]     CERTIFICATE_VERIFY_FAILED] certificate verify failed:
[nltk_data]     unable to get local issuer certificate (_ssl.c:992)>
[nltk_data] Error loading punkt: <urlopen error [SSL:
[nltk_data]     CERTIFICATE_VERIFY_FAILED] certificate verify failed:
[nltk_data]     unable to get local issuer certificate (_ssl.c:992)>


False

In [4]:
df = pd.read_csv("/Users/harmyabhatt/TextMiningAndInjuryClassification/dataFiles/model_dataset.csv")
display(df)

,text,label
0,excellent for beginners. the board is sturdy b...,0
1,i am a year old man thinking i can skate agai...,0
2,i am a year old man thinking i can skate agai...,0
3,muy buena,0
4,article conforme à la description. a l'air sol...,0
...,...,...
31508,my daughter loves this skateboard and goes out...,0
31509,i am a year old man thinking i can skate agai...,0
31510,pt riding his trick scooter when he hit the cu...,1
31511,wir haben unserem enkel diesen roller geschenk...,0


In [5]:
print(df['label'].value_counts())
df = df.rename(columns={'Final Narrative': 'text', 'Event': 'label'})
df['label'] = df['label'].replace({42: 'Fall on Same Level', 43:'Fall to Lower Level'})
df

0    23723
1     7790
Name: label, dtype: int64


,text,label
0,excellent for beginners. the board is sturdy b...,0
1,i am a year old man thinking i can skate agai...,0
2,i am a year old man thinking i can skate agai...,0
3,muy buena,0
4,article conforme à la description. a l'air sol...,0
...,...,...
31508,my daughter loves this skateboard and goes out...,0
31509,i am a year old man thinking i can skate agai...,0
31510,pt riding his trick scooter when he hit the cu...,1
31511,wir haben unserem enkel diesen roller geschenk...,0


In [6]:
#making dataset and preprocessing
dataset_text = list(df.text)
dataset_labels = list(df.label)
text = []
labels = []
dataset = pd.DataFrame()
for i in range(0, len(dataset_text)):
    t = str(dataset_text[i])
    l = dataset_labels[i]

    if 0 <= len(str(t).split(" ")) <= 40:
        words = str(t).split(" ")
        t = re.sub('lt', 'left', t)
        t = re.sub('lwr', 'lower', t)
        t = re.sub('rt', 'right', t)
        t = re.sub('pt', ' ', t)
        t = re.sub('c/o', ' ', t)
        t = re.sub('dx', ' ', t)
        t = re.sub(r'[^a-zA-Z ]+', ' ', t)
        t = re.sub(' +', ' ', t)
        text.append(t)
        labels.append(l)

dataset['text'] = text
dataset['label'] = labels

dataset = dataset.groupby('label', group_keys=False).apply(lambda x: x.sample(7000)).reset_index(drop=True)
dataset = dataset.sample(frac=1)
print(dataset['label'].value_counts())
display(dataset)

1    7000
0    7000
Name: label, dtype: int64


,text,label
13217,skateboarding when he fell forward onto outstr...,1
3748,i pulled this out of the box to check it out t...,0
1562,why rent when you can buy and once you ride yo...,0
1423,my four year old loves it,0
9200,was riding a scooter downhill at a park when h...,1
...,...,...
4628,nice,0
9800,presents with digit injury was lying prone on ...,1
11409,fell off a skateboard and landed on outstretch...,1
4617,der kinder roller ist sehr gut gebaut sehr sta...,0


### Vectorizer For Model Training

In [7]:
# split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(dataset["text"], dataset["label"], test_size=0.2, shuffle=False)

# create bag-of-words with weights using tfid vectoriser
# strip accents and remove stop words during vectorisation
tf = TfidfVectorizer(max_features=10000, ngram_range=(1,2))
cv = CountVectorizer(max_features=10000, ngram_range=(1,2))

# transform and fit the training set with vectoriser
X_train_tf = tf.fit_transform(X_train)
# transform the test set with vectoriser
x_test_tf = tf.transform(X_test)


# # create logistic regression model
# gnb = GaussianNB()
# lreg = LinearRegression()
# logreg = LogisticRegression(solver='newton-cg')
# random_forest = RandomForestRegressor(n_estimators=100)
# # train model on  vectorised training data
# #model_linear = lreg.fit(X_train_tf, y_train)
# #model_logistic = logreg.fit(X_train_tf, y_train)
# #clf = MultinomialNB().fit(X_train_tf, y_train)
# rf = random_forest.fit(X_train_tf, y_train)

# # evaluate model performance on the test set
# pred_rf = rf.predict(prediction)
# #pred_linear = model_linear.predict(prediction)
# pred_labels = []
# for value in pred_rf:
#     if value > 0.5:
#         pred_labels.append(1)
#     else:
#         pred_labels.append(0)

# #pred_logistic = model_logistic.predict(prediction)
# #pred_naive = clf.predict(prediction)
# #print(sklearn.metrics.accuracy_score(y_test, pred_logistic))
# print(sklearn.metrics.classification_report(y_test, pred_labels, digits=5))

### Linear Regression

In [8]:
lreg = LinearRegression()
model_linear = lreg.fit(X_train_tf, y_train)
pred_linear = model_linear.predict(x_test_tf)
pred_labels = []
for value in pred_linear:
    if value > 0.5:
        pred_labels.append(1)
    else:
        pred_labels.append(0)

print(sklearn.metrics.classification_report(y_test, pred_labels, digits=5))

              precision    recall  f1-score   support

           0    0.79170   0.81272   0.80207      1431
           1    0.79865   0.77648   0.78741      1369

    accuracy                        0.79500      2800
   macro avg    0.79517   0.79460   0.79474      2800
weighted avg    0.79509   0.79500   0.79490      2800



### Logistic Regression

In [11]:
# logistic regression model

logreg = LogisticRegression(solver='newton-cg')
model_logistic = logreg.fit(X_train_tf, y_train)
pred_logistic = model_logistic.predict(x_test_tf)
pred_labels = []
for value in pred_logistic:
    if value > 0.5:
        pred_labels.append(1)
    else:
        pred_labels.append(0)

print(sklearn.metrics.classification_report(y_test, pred_labels, digits=5))

              precision    recall  f1-score   support

           0    0.99860   0.99860   0.99860      1431
           1    0.99854   0.99854   0.99854      1369

    accuracy                        0.99857      2800
   macro avg    0.99857   0.99857   0.99857      2800
weighted avg    0.99857   0.99857   0.99857      2800



### Naive-Bayes Model

In [12]:
#naive bayes model

clf = MultinomialNB().fit(X_train_tf, y_train)
pred_naive = clf.predict(x_test_tf)
pred_labels = []
for value in pred_naive:
    if value > 0.5:
        pred_labels.append(1)
    else:
        pred_labels.append(0)

print(sklearn.metrics.classification_report(y_test, pred_labels, digits=5))

              precision    recall  f1-score   support

           0    1.00000   0.99371   0.99685      1431
           1    0.99347   1.00000   0.99672      1369

    accuracy                        0.99679      2800
   macro avg    0.99673   0.99686   0.99678      2800
weighted avg    0.99681   0.99679   0.99679      2800



### Random Forest Regressor

In [13]:
#random forest model


random_forest = RandomForestRegressor(n_estimators=100)
rf = random_forest.fit(X_train_tf, y_train)
pred_rf = rf.predict(x_test_tf)
pred_labels = []
for value in pred_rf:
    if value > 0.5:
        pred_labels.append(1)
    else:
        pred_labels.append(0)

print(sklearn.metrics.classification_report(y_test, pred_labels, digits=5))

              precision    recall  f1-score   support

           0    0.99718   0.98952   0.99334      1431
           1    0.98913   0.99708   0.99309      1369

    accuracy                        0.99321      2800
   macro avg    0.99316   0.99330   0.99321      2800
weighted avg    0.99325   0.99321   0.99321      2800

